In [0]:
# Create model function for Keras Tuner to search hyperparameter.
num_classes = 2
LOG_DIR = "eason_kt_"+f"{int(time.time())}"

def build_model(hp):  # random search passes this hyperparameter() object.
    model = keras.models.Sequential()
    model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 1)))

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        model.add(layers.Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), 1,
                                padding='same'))
        model.add(layers.Activation('relu'))
        model.add(layers.MaxPooling2D())
        model.add(layers.Dropout(hp.Choice('dropout_value', values=[0.2, 0.5])))
        
    model.add(layers.Flatten()) 
    #model.add(layers.Dense(128))
    model.add(layers.Dense(hp.Int('dense_units',
                               min_value=32,
                               max_value=128,
                               step=32)))
    #model.add(layers.Activation("relu"))
    model.add(layers.Activation(hp.Choice(name='last_activation',
                                          values=['relu', 'sigmoid'],
                                          default='relu'))) # adding variation of last activation.
    model.add(layers.Dense(num_classes))
    
    #optimizer=hp.Choice(name='optimizer', values=['adam', 'sgd', 'rmsprop']) # adding variation of optimizer.
    #hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) # adding variation of learning rate.
    hp_learning_rate = hp.Float('learning_rate',
                                min_value=1e-4,
                                max_value=1e-2,
                                sampling='LOG', # log will assign equal probabilities to each order of magnitude range.
                                default=1e-3) # adding range variation of learning rate for adam.
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return model